In [ ]:
import pandas as pd
import numpy as np
import sklearn as sk

# from sklearn.preprocessing import MinMaxScaler
# from sklearn.model_selection import train_test_split
import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense


## loading the dataset and converting to standard types

In [ ]:
df = pd.read_csv("train/eth.csv")
df["timestamp"] = pd.to_datetime(df["timestamp"])
df = df.sort_values(by="timestamp").reset_index(drop=true)
df.interpolate(inplace=true)
print(len(df))

595814


## feature engineering
- adding new features and deciding which feature really corellates to the volatility

In [ ]:
# may have to add a few more


# spread
df["spread"] = df["ask_price1"] - df["bid_price1"]

# mid price change
df["mid_price_change"] = df["mid_price"].diff()

# imbalance
bid_volume_sum = df[[f"bid_volume{i}" for i in range(1, 6)]].sum(axis=1)
ask_volume_sum = df[[f"ask_volume{i}" for i in range(1, 6)]].sum(axis=1)
df["imbalance"] = (bid_volume_sum - ask_volume_sum) / (bid_volume_sum + ask_volume_sum)

# bid weighted average price
df["bid_dwap"] = (
    df[[f"bid_price{i}" for i in range(1, 6)]].values
    * df[[f"bid_volume{i}" for i in range(1, 6)]]
).sum(axis=1)

# ask weighted average price
df["ask_dwap"] = (
    df[[f"ask_price{i}" for i in range(1, 6)]].values
    * df[[f"ask_volume{i}" for i in range(1, 6)]]
).sum(axis=1)

# rolling volume
df["rolling_vol"] = df["mid_price"].rolling(window=5).std()

# 3. Spread-Weighted Liquidity
near_liq_bid = sum(
    df[f"bid_volume{i}"] / (df["mid_price"] - df[f"bid_price{i}"]) for i in range(1, 6)
)
near_liq_ask = sum(
    df[f"ask_volume{i}"] / (df[f"ask_price{i}"] - df["mid_price"]) for i in range(1, 6)
)
df["Near_Liquidity"] = near_liq_bid + near_liq_ask

# 4. Microprice
df["Microprice"] = (
    df["ask_price1"] * df["bid_volume1"] + df["bid_price1"] * df["ask_volume1"]
) / (df["bid_volume1"] + df["ask_volume1"])

# 5. Order Book Slope
df["Bid_Slope"] = (df["bid_price5"] - df["bid_price1"]) / bid_volume_sum
df["Ask_Slope"] = (df["ask_price5"] - df["ask_price1"]) / ask_volume_sum

# 6. Short-Term Imbalance Change
df["Delta_OBI"] = df["imbalance"].diff()

In [ ]:
# X_COL = ["spread", "mid_price_change", "imbalance", "mid_price", "bid_dwap", "ask_dwap", "rolling_vol", "Near_Liquidity", "Microprice", "Bid_Slope", "Ask_Slope", "Delta_OBI"]
X_COL = [
    "spread",
    "mid_price_change",
    "imbalance",
    "mid_price",
    "bid_dwap",
    "ask_dwap",
    "rolling_vol",
    "Near_Liquidity",
    "Microprice",
    "Bid_Slope",
    "Ask_Slope",
    "Delta_OBI",
]
TARGET_COL = "label"
X = df[X_COL]
y = df[TARGET_COL]

In [1]:
# For demonstration, let's assume you already have X, y
# Scale features
scaler_X = sk.preprocessing.MinMaxScaler()
scaler_y = sk.preprocessing.MinMaxScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1))

# Train/test split
X_train, X_test, y_train, y_test = sk.model_selection.train_test_split(
    X_scaled, y_scaled, test_size=0.2, shuffle=False
)

# Reshape for LSTM: (samples, timesteps, features)
# Here timesteps=1 because we treat each row as a single step
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# Build LSTM model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(tf.keras.layers.Dense(1))

model.compile(optimizer="adam", loss="mse")

# Train
history = model.fit(
    X_train,
    y_train,
    epochs=20,
    batch_size=64,
    validation_data=(X_test, y_test),
)

# Predict
y_pred_scaled = model.predict(X_test)
y_pred = scaler_y.inverse_transform(y_pred_scaled)
y_true = scaler_y.inverse_transform(y_test)

# Evaluate
mae = np.mean(np.abs(y_pred - y_true))
print("MAE:", mae)


NameError: name 'sk' is not defined